In [1]:
from dask.distributed import Client, progress
client = Client(processes=False, threads_per_worker=4,
                n_workers=1, memory_limit='2GB')
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://172.29.12.79:8787/status,
Dashboard: http://172.29.12.79:8787/status,Workers: 1
Total threads: 4,Total memory: 1.86 GiB
Status: running,Using processes: False
Comm: inproc://172.29.12.79/4137/1,Workers: 0
Dashboard: http://172.29.12.79:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: inproc://172.29.12.79/4137/4,Total threads: 4
Dashboard: http://172.29.12.79:39337/status,Memory: 1.86 GiB
Nanny: None,
